In [180]:
import numpy as np
import cvxpy as cp 
from queue import Queue
from copy import deepcopy
from time import sleep

In [181]:
A = np.matrix([[1, 2, 3],[3, 4, 3],[2, 1, 0]])
B = np.matrix([[1, 2, 3],[3, 4, 3],[2, 1, 0]])
x = np.array([1,0,0.5])
y = np.matrix([3,4,5]).T
np.linalg.norm(x)
np.count_nonzero(x)
lambda_0 = 1
M0 = 10000
A @ B

matrix([[13, 13,  9],
        [21, 25, 21],
        [ 5,  8,  9]])

In [182]:
A @ x

matrix([[2.5, 4.5, 2. ]])

# We want to solve the following

$$\min_{x \in \real^n} \frac{1}{2} ||y - A x||^2_2 + \lambda ||x||_0$$

In [183]:
def evaluation_main (y, A, x, alpha = lambda_0):
    value = 0.5 * np.linalg.norm( y - A @ x ) ** 2 + alpha * np.count_nonzero(x)    
    return value

In [184]:
evaluation_main(y, A, x)

14.749999999999998

# We are going to create a BnB of the following

$$\min_{x \in \real^n} \frac{1}{2} ||y - A x||^2_2 + \frac{\lambda}{M} ||x_{\bar{S}}||_1 + \lambda |S_1|$$

In [185]:
np.random.seed(421)
A = np.random.randint(-10, 10, (10, 10))
y = np.random.randint(-10, 10, (10, 1))
n = y.shape[0]
n

10

In [186]:
lambda_0 = 1
M0 = 100


def evaluation_main (y, A, x, alpha = lambda_0):
    value = 0.5 * np.linalg.norm( y - A @ x ) ** 2 + alpha * np.count_nonzero(x)    
    return value


def check_bound(u, v):
    if v.pu <= u.pu:
        return v.pu, v
    
    else:
        return u.pu, u
    



def relax_problem (y, A, S0, S1, S, alpha = lambda_0, M = M0):
    n = y.shape[0]
    X = cp.Variable((n, 1))
    obj =  0.5 * cp.sum_squares(y - A @ X ) 
    obj +=  alpha / M * cp.norm(X[S], 1)
    obj += alpha * len(S1)

    
    if len(S0) == 0:
        constraint = [X <= M, -X <= M] 
    else:
        constraint = [X[S0] == 0, X <= M, -X <= M] 

    prob = cp.Problem(cp.Minimize( obj ), constraint)
    result = prob.solve(verbose = False, solver = cp.ECOS, abstol=1e-4, reltol=1e-4)
    
    #eliminate 0
    x_vector = X.value
    x_vector = np.where(abs(x_vector) < 1e-7, 0, x_vector )
    return result, x_vector



class Node:
    
    def __init__(self, n, S0, S1, S, A, y):
        self.level = n - len(S)
        self.A = A
        self.y = y
        self.S0 = S0
        self.S1 = S1
        self.S = S
        self.pvl = 0
        self.pu = None       #corresponds to obj func value in real problem with x
        self.x = None
        self.calculate_pvl()
        self.calculate_pu_x()
        
    
    def calculate_pvl (self) -> float:
        self.pvl, self.x = relax_problem(y, A, S0 = self.S0, S1 = self.S1, S = self.S)
        return self.pvl
        
    def calculate_pu_x (self) -> float:
        self.pu =  evaluation_main(y, A, self.x)
        return self.pu
        

    def __le__ (self, other) -> bool:
        return self.pvl <= other.pvl
    
    def __str__(self) -> str:
        text = f"S0: {self.S0} | S1: {self.S1} | S: {self.S}\nPvl: {self.pvl}\n\n"
        return text
    
    def __repr__(self) -> str:
        text = f"S0: {self.S0} | S1: {self.S1} | Pvl: {self.pvl}"
        return text

In [187]:
S = list(range(n))
S0 = []
S1 = []
u = Node(n, S0, S1, S)

In [188]:
S = list(range(n))
S0 = []
S1 = []

u1 = Node(n, S0, S1, S)
print(u1.x)
print(u1.calculate_pvl())
print(u1.calculate_pu_x())

S = deepcopy(u.S)
S1 = deepcopy(u.S1)
S0 = deepcopy(u.S0)

index = S.pop(0)
index2 = S.pop(0)

#None zero branch
S0.append(index)
S0.append(index2)

u2 = Node(n, S0, S1, S)

print(u1 >= u2)
print(u1.pu, u2.pu)
check_bound(u1, u2)

[[ 0.23170507]
 [-0.43967887]
 [-0.81020599]
 [ 0.62298317]
 [ 0.09508656]
 [ 0.45371026]
 [-0.08893891]
 [ 0.82366556]
 [-0.06056742]
 [-1.4987601 ]]
0.05135416567501372
10.000101146445013
False
10.000101146445013 22.805888336600418


(10.000101146445013, S0: [] | S1: [] | Pvl: 0.05135416567501372)

## We are going to create the order

In [189]:
S = list(range(n))
S0 = []
S1 = []

#crear queue
q = Queue()
u = Node(n, S0, S1, S)
q.put(u)

#optimum 
pv_opt = u.pu
node_opt = deepcopy(u)
print("first op:", pv_opt)




while not q.empty():
    
    u = q.get()
    print(f"u node: S0: {u.S0} | S1: {u.S1} | Pvl: {u.pvl}")
    
    #if the node is leaf
    if u.level == n  or len(u.S) == 0:
        continue
    
    S_v = deepcopy(u.S)
    S1_v = deepcopy(u.S1)
    S0_v = deepcopy(u.S0)
    
    index = S_v.pop(0)
    
    #None zero branch
    S1_v.append(index)
    
    v1 = Node(n, S0_v, S1_v, S_v)
    print("v1 pvl: ", v1.pvl)
    if not pv_opt <= v1.pvl:
        q.put(v1)
        print(v1.pu)             
        pv_opt, node_opt = check_bound(node_opt, v1)

        
    
    #Zero branch
    S_v = deepcopy(u.S)
    S1_v = deepcopy(u.S1)
    S0_v = deepcopy(u.S0)
    index = S_v.pop(0)
    S0_v.append(index)
    S1_v = deepcopy(u.S1)
    v0 = Node(n, S0_v, S1_v, S_v)
    print("V0 pvl:", v0.pvl)
    if not pv_opt <= v0.pvl:
        q.put(v0)
        print(v0.pu)             
        pv_opt, node_opt = check_bound(node_opt, v0)
    
    print("Queue:\t",list(q.queue), "\n")

    

print("Opt obj value:\t", pv_opt)
print("Optimum node:\n", node_opt ) 

first op: 10.000101146445013
u node: S0: [] | S1: [] | Pvl: 0.05135416567501372
v1 pvl:  1.0490305868541316
10.000063112047783
V0 pvl: 0.2519252116059567
9.207988171831724
Queue:	 [S0: [] | S1: [0] | Pvl: 1.0490305868541316, S0: [0] | S1: [] | Pvl: 0.2519252116059567] 

u node: S0: [] | S1: [0] | Pvl: 1.0490305868541316
v1 pvl:  2.044623951208538
10.000047815949076
V0 pvl: 3.1371708063922306
11.099604311844722
Queue:	 [S0: [0] | S1: [] | Pvl: 0.2519252116059567, S0: [] | S1: [0, 1] | Pvl: 2.044623951208538, S0: [1] | S1: [0] | Pvl: 3.1371708063922306] 

u node: S0: [0] | S1: [] | Pvl: 0.2519252116059567
v1 pvl:  1.248845178440732
9.207988660130608
V0 pvl: 14.854662245513904
Queue:	 [S0: [] | S1: [0, 1] | Pvl: 2.044623951208538, S0: [1] | S1: [0] | Pvl: 3.1371708063922306, S0: [0] | S1: [1] | Pvl: 1.248845178440732] 

u node: S0: [] | S1: [0, 1] | Pvl: 2.044623951208538
v1 pvl:  3.0365114934775246
10.000028746516655
V0 pvl: 8.235475176911601
15.205751148022994
Queue:	 [S0: [1] | S1: [0]

V0 pvl: 9.800036848011835
Queue:	 [S0: [1] | S1: [0, 2] | Pvl: 4.131656120475425, S0: [1, 2] | S1: [0] | Pvl: 7.244039609690718, S0: [0] | S1: [1, 2] | Pvl: 2.241620846789979, S0: [0, 2] | S1: [1] | Pvl: 8.955488283338461, S0: [] | S1: [0, 1, 2, 3] | Pvl: 4.030268578879636] 

u node: S0: [1] | S1: [0, 2] | Pvl: 4.131656120475425
v1 pvl:  5.1286229639399625
11.099593476711618
V0 pvl: 13.71458702828024
Queue:	 [S0: [1, 2] | S1: [0] | Pvl: 7.244039609690718, S0: [0] | S1: [1, 2] | Pvl: 2.241620846789979, S0: [0, 2] | S1: [1] | Pvl: 8.955488283338461, S0: [] | S1: [0, 1, 2, 3] | Pvl: 4.030268578879636, S0: [1] | S1: [0, 2, 3] | Pvl: 5.1286229639399625] 

u node: S0: [1, 2] | S1: [0] | Pvl: 7.244039609690718
v1 pvl:  8.241897529293094
14.213992783157078
V0 pvl: 13.22536442509883
Queue:	 [S0: [0] | S1: [1, 2] | Pvl: 2.241620846789979, S0: [0, 2] | S1: [1] | Pvl: 8.955488283338461, S0: [] | S1: [0, 1, 2, 3] | Pvl: 4.030268578879636, S0: [1] | S1: [0, 2, 3] | Pvl: 5.1286229639399625, S0: [1, 2